In [8]:
import pandas as pd
from hdfs.client import Client

# читаем с HDFS


def read_hdfs_file(client, filename):
    lines = []
    with client.read(filename, encoding='utf-8', delimiter='\n') as reader:
        for line in reader:
            lines.append(line.strip())
    return lines

# Записываем данные в файл hdfs


def write_to_hdfs(client, hdfs_path, data):
    client.write(hdfs_path, data, overwrite=True, encoding='utf-8')

# загружаем файлы из hdfs на локальный (можете попробовать из джупитера, если установите библиотеку hdfs)


def get_from_hdfs(client, hdfs_path, local_path):
    client.download(hdfs_path, local_path, overwrite=False)

# Загружаем файл в hdfs


def put_to_hdfs(client, local_path, hdfs_path):
    client.upload(hdfs_path, local_path, cleanup=True)


# инициализируем HDFS клиент и создаем подключение к адресу HDFS
client = Client("http://new-adh-master-1.ru-central1.internal:9870",
                root="/", timeout=5, session=False)

data = read_hdfs_file(client, '/tmp/ai/names/test.csv')

#write_to_hdfs(client, "/mlengineer/kuznetsov_k/data.csv", '\n'.join(data))

ConnectionError: HTTPConnectionPool(host='new-adh-node-1.ru-central1.internal', port=9864): Max retries exceeded with url: /webhdfs/v1/tmp/ai/names/test.csv?op=OPEN&namenoderpcaddress=new-adh-master-1.ru-central1.internal:8020&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15e5da250>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [9]:
client

<Client(url='http://new-adh-master-1.ru-central1.internal:9870')>

---

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df = pd.DataFrame({
    'one': [-1, np.nan, 2.5],
    'two': ['foo', 'bar', 'baz'],
    'three': [True, False, True]},
    index=list('abc'))

table = pa.Table.from_pandas(df)
pq.write_table(table, './data/example.parquet')

In [6]:
import avro
import json
from avro.io import DatumWriter, DatumReader
from avro.datafile import DataFileWriter, DataFileReader

# Задаем схему данных для файла
schema = {
    'name': 'avro.example.User',
    'type': 'record',
    'fields': [
        {'name': 'name', 'type': 'string'},
        {'name': 'age', 'type': 'int'}
    ]
}

# парсим схему, чтобы использовать ее для записи данных
schema_parsed = avro.schema.parse(json.dumps(schema))

# Пишем данные в avro файл
with open('./data/users.avro', 'wb') as f:
    writer = DataFileWriter(f, DatumWriter(), schema_parsed)
    writer.append({'name': 'Иван', 'age': 35})
    writer.append({'name': 'Александр', 'age': 66})
    writer.close()
